# railway site automation

In [2]:
# Step 1: Import required libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime, timedelta
import time
from selenium.webdriver.common.by import By
import requests
import io
import os
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment
from openpyxl import load_workbook
import yagmail

In [3]:
# Step 2: Start Chrome and open the website
chrome_path = r"C:\Path\To\chromedriver.exe"  # 👉 Replace with your actual path to chromedriver

chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox") 

driver = webdriver.Chrome(options=chrome_options)

# Open the target URL
driver.get("https://www.ireps.gov.in/epsn/anonymSearchPO.do?searchParam=showPageSupply&language=en")

# Wait for page to fully load
time.sleep(4)

In [4]:
# ✅ Refresh the current tab (useful if data loads dynamically or expires)
driver.refresh()

# Wait again after refresh
time.sleep(3)

In [5]:
# Step 3: Fill in the PO value range
po_value_from = driver.find_element(By.ID, "poValueFrom")
po_value_to = driver.find_element(By.ID, "poValueTo")

# Clear and enter the values
po_value_from.clear()
po_value_from.send_keys("1")

po_value_to.clear()
po_value_to.send_keys("99999")

In [27]:
# Step 1: Calculate yesterday's date in dd/mm/yyyy format
yesterday = datetime.today() - timedelta(days=1)
yesterday_str = yesterday.strftime("%d/%m/%Y")
print(yesterday_str)

20/05/2025


In [57]:
# Step 3: Click the calendar icon
calendar_icon = driver.find_element(By.XPATH, "//img[contains(@src,'cal.gif')]")
calendar_icon.click()

# Step 4: Wait for new window and switch to it
WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
main_window = driver.current_window_handle
all_windows = driver.window_handles

for window in all_windows:
    if window != main_window:
        driver.switch_to.window(window)
        break

# Step 5: Wait for calendar to load and select yesterday’s date
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//td[contains(@onclick, \"callback('ddmmyyDateformat1','{yesterday_str}')\")]"))
)

# Step 6: Click yesterday’s date
driver.find_element(By.XPATH, f"//td[contains(@onclick, \"callback('ddmmyyDateformat1','{yesterday_str}')\")]").click()

In [58]:
# Step 7: Switch back to the main window
driver.switch_to.window(main_window)

In [59]:
# Step 3: Click second calendar icon (for ddmmyyDateformat2)
calendar_icon_2 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, \"NewCal('ddmmyyDateformat2'\")]/img"))
)
calendar_icon_2.click()

# Step 4: Wait for popup window and switch to it
WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
for handle in driver.window_handles:
    if handle != main_window:
        driver.switch_to.window(handle)
        break

# Step 5: Wait and click the date (yesterday)
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, f"//td[contains(@onclick, \"callback('ddmmyyDateformat2','{yesterday_str}')\")]"))
).click()

In [60]:
# Step 6: Switch back to main window
driver.switch_to.window(main_window)

In [10]:
# ✅ Step 4: Extract first 6 characters of captcha text from image URL (no OCR needed)

# Find the captcha image element
captcha_img = driver.find_element(By.XPATH, '//img[contains(@src,"Captcha.jpg")]')
captcha_url = captcha_img.get_attribute('src')

# Extract the captcha text from the 'r=' parameter
import urllib.parse as urlparse

parsed = urlparse.urlparse(captcha_url)
full_captcha = urlparse.parse_qs(parsed.query).get('r', [''])[0]  # Get the 'r' value

# Get only first 6 characters from the left
captcha_text = full_captcha[:6]

print("✅ Captcha Text (first 6 characters):", captcha_text)

✅ Captcha Text (first 6 characters): 17739c


In [11]:
# Step 5: Fill in captcha text and submit
# Find the captcha input field
captcha_input = driver.find_element(By.XPATH, '//input[@id="verification"]')

# Fill the captcha input field with the captcha text
captcha_input.send_keys(captcha_text)

In [12]:
# Find the submit input element by its XPath
submit_element = driver.find_element(By.XPATH, '//input[@type="submit" and @name="submit" and @value="Show Results"]')

submit_element.click()
# Click using JavaScript to ensure JS events fire
            
# Wait for 9 seconds for the results to load or any action to complete
time.sleep(9)

print("✅ Form submitted successfully, and waited for 9 seconds.")

✅ Form submitted successfully, and waited for 9 seconds.


let's print the total results of yesterday

In [14]:
# ✅ Find all <td class='formLabel'> elements
try:
    td_elements = driver.find_elements(By.XPATH, "//td[@class='formLabel']")
    print(f"🔍 Found {len(td_elements)} <td class='formLabel'> elements.")
    
    # ✅ Loop through all <td> elements and find the one containing 'Tender search results'
    for index, td in enumerate(td_elements):
        inner_html = td.get_attribute("innerHTML")
        
        if 'Tender search results' in inner_html:
            print(f"\n🟨 Found the target <td> at index {index}.")
            b_tags = td.find_elements(By.TAG_NAME, "b")
            
            if len(b_tags) >= 2:
                result_count = b_tags[1].text.strip()
                print(f"✅ Tender search result count: {result_count}")
                print(f"💡 Total results of {yesterday_str} is {result_count}")
            else:
                print(f"❌ Expected at least 2 <b> tags, but found {len(b_tags)}.")
            break
    else:
        print("❌ 'Tender search results' not found in any <td> elements.")
        
except Exception as e:
    print("⚠️ Error occurred:", e)

🔍 Found 24 <td class='formLabel'> elements.

🟨 Found the target <td> at index 14.
✅ Tender search result count: 696
⚠️ Error occurred: name 'yesterday_str' is not defined


In [15]:
# Step 1: After form submission, get HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Step 2: Locate the main table
table = soup.find('table', {'border': '1', 'bordercolor': '#4D817A'})

# Step 3: Extract all rows
rows = table.find_all('tr')[1:]  # skip header

# Step 4: Prepare final data list
data = []
current_info = {}  # to hold Sr, Supplier, PO Value, Description

for row in rows:
    cells = row.find_all('td')
    
    if len(cells) >= 9:
        # This is the first/main row (with Sr, Supplier, PO Value, etc.)
        current_info = {
            'Sr': cells[0].get_text(strip=True),
            'Supplier Name': cells[1].get_text(strip=True),
            'PO Value': cells[2].get_text(strip=True),
            'Item Description': cells[3].get_text(strip=True),
        }

        # Now append first row
        data.append({
            'Sr': current_info['Sr'],
            'Supplier Name': current_info['Supplier Name'],
            'PO Value': current_info['PO Value'],
            'Item Description': current_info['Item Description'],
            'PO Sl': cells[4].get_text(strip=True),
            'Consignee': cells[5].get_text(strip=True),
            'PO Qty/Unit': cells[6].get_text(strip=True),
            'T.U.R.': cells[7].get_text(strip=True),
            'Dely.Dt': cells[8].get_text(strip=True)
        })

    elif len(cells) == 5:
        # These are child rows - just PO Sl onwards
        data.append({
            'Sr': '',
            'Supplier Name': '',
            'PO Value': '',
            'Item Description': '',
            'PO Sl': cells[0].get_text(strip=True),
            'Consignee': cells[1].get_text(strip=True),
            'PO Qty/Unit': cells[2].get_text(strip=True),
            'T.U.R.': cells[3].get_text(strip=True),
            'Dely.Dt': cells[4].get_text(strip=True)
        })

In [16]:
# Step 5: Convert to DataFrame
df = pd.DataFrame(data)

In [17]:
df.head(10)

,Sr,Supplier Name,PO Value,Item Description,PO Sl,Consignee,PO Qty/Unit,T.U.R.,Dely.Dt
0,1,SER/EF255002100014 dt. 04-MAR-2025 on M/s. NAR...,395598.96,660114760014: Spares for Electric Air Compress...,001,SSE(C&W)BNDM,1 Set,395598.96,10-MAY-2025
1,2,ICF/04241075100346 dt. 04-MAR-2025 on M/s. DEB...,179902.8,77110043: SUPPLY AND APPLICATION OF PAINT ON B...,001,SHELL,66 Set,2725.80,30-SEP-2025
2,3,SECR/08251032200601 dt. 04-MAR-2025 on M/s. RA...,1057980,80014793: actevo 210 Litres Internal Combustio...,001,GSD/R,1470 LTR,114.50,23-APR-2025
3,,,,,002,GSD/R,4620 LTR,114.50,22-JUL-2025
4,,,,,003,GSD/R,3150 LTR,114.50,29-NOV-2025
5,4,ICF/01243144100348 dt. 04-MAR-2025 on M/s. AMA...,206293.5,72212974: THROW AWAY INSERTS LNMX 301940 PM 43...,001,SHELL,150 Nos.,1375.29,04-JUN-2025
6,5,ICF/01241267100350 dt. 04-MAR-2025 on M/s. PIO...,426275,73853872: HEX SOCKET HEAD CAP SCREW SIZE M12 X...,001,LHB,42500 Nos.,10.03,30-JUN-2025
7,6,SECR/08241006100602 dt. 04-MAR-2025 on M/s. PR...,3103332.6,80010635: Shell Carnea-41/Oil Machinery/Genera...,001,GSD/R,3546 LTR,77.22,05-MAY-2025
8,,,,,002,GSD/R,3400 LTR,77.22,31-OCT-2025
9,,,,,003,RWSS/R,2613 LTR,77.22,05-MAY-2025


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Sr                1259 non-null   object
 1   Supplier Name     1259 non-null   object
 2   PO Value          1259 non-null   object
 3   Item Description  1259 non-null   object
 4   PO Sl             1259 non-null   object
 5   Consignee         1259 non-null   object
 6   PO Qty/Unit       1259 non-null   object
 7   T.U.R.            1259 non-null   object
 8   Dely.Dt           1259 non-null   object
dtypes: object(9)
memory usage: 88.7+ KB


set path

save data as csv

In [29]:
# 📌 Your desired folder path (replace this with your actual path)
folder_path = r"c:\users\solan\python_things\railway_data"  # <-- Paste your full folder path here

# ✅ Clean the date string
clean_date = yesterday_str.replace('/', '_')  # e.g., '12052025'

# ✅ Create the filename
filename = f"{clean_date}_results_{result_count}.xlsx"

# ✅ Full path to save
full_path = os.path.join(folder_path, filename)

# ✅ Save the CSV file
df.to_excel(full_path, index=False)

print(f"✅ Data saved at: {full_path}")
print(f"✅ filename: {filename}")
print(f"✅ file_Path: {full_path}")

✅ Data saved at: c:\users\solan\python_things\railway_data\20_05_2025_results_696.xlsx
✅ filename: 20_05_2025_results_696.xlsx
✅ file_Path: c:\users\solan\python_things\railway_data\20_05_2025_results_696.xlsx


In [31]:
# ✅ Load workbook for formatting
wb = load_workbook(full_path)
ws = wb.active

# Add header to new column (next to last column)
ws.cell(row=1, column=ws.max_column + 1).value = "Direction/Location"

# Fill values based on Supplier Name
for row in range(2, ws.max_row + 1):
    supplier_name = ws.cell(row=row, column=2).value  # Column B = Supplier Name
    if supplier_name and "/" in supplier_name:
        direction = supplier_name.split("/")[0]
        ws.cell(row=row, column=ws.max_column).value = direction
    else:
        ws.cell(row=row, column=ws.max_column).value = ""

# ✅ Freeze top row
ws.freeze_panes = "A2"

# ✅ Set zoom to 100%
ws.sheet_view.zoomScale = 95

# ✅ Column Widths (as you provided)
column_widths = {
    'A': 6.8,# Sr
    'B': 35,     # Supplier Name
    'C': 14, # PO Value
    'D': 62,     # Item Description
    'E': 7.5,    # PO Sl
    'F': 15.6,   # Consignee
    'G': 18,  # PO Qty/Unit
    'H': 12.8,   # T.U.R.
    'I': 19.5, #dely.dt
    'J': 25, #direction/location which is new column
}

for col_letter, width in column_widths.items():
    ws.column_dimensions[col_letter].width = width

# ✅ Apply header font: size 14 + bold + center align
for cell in ws[1]:
    cell.font = Font(size=14, bold=True)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# ✅ Apply formatting to all data rows
for row in ws.iter_rows(min_row=2, max_row=ws.max_row, max_col=ws.max_column):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")

# ✅ Wrap Text for 'Supplier Name' (B) and 'Item Description' (D)
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    row[1].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")  # B
    row[3].alignment = Alignment(wrap_text=True, horizontal="center", vertical="center")  # D

# ✅ Format 'Dely.Dt' column (I) as "14-March-2025"
for row in ws.iter_rows(min_row=2, min_col=9, max_col=9):
    for cell in row:
        try:
            cell.value = pd.to_datetime(cell.value).date()
            cell.number_format = 'DD-MMMM-YYYY'
        except:
            continue

# Apply filter to top row A1 to last column
last_col_letter = get_column_letter(ws.max_column)
ws.auto_filter.ref = f"A1:{last_col_letter}1"

print("Column added and filter applied, and all other foramating too")

Column added and filter applied, and all other foramating too


In [32]:
#save information in excel
wb.save(full_path)
print(f"✅ Final Excel saved at: {full_path}")
print(f"✅ filename: {filename}")
print(f"✅ file_Path: {full_path}")

✅ Final Excel saved at: c:\users\solan\python_things\railway_data\20_05_2025_results_696.xlsx
✅ filename: 20_05_2025_results_696.xlsx
✅ file_Path: c:\users\solan\python_things\railway_data\20_05_2025_results_696.xlsx


In [33]:
#yogmail
#to edit or check app password varify your email with this 'https://myaccount.google.com/apppasswords'

sender_email = "sender email"
app_password = "your generated mail app password"  # Replace with the app password
receiver_emails = ["receiver's email"]
subject = "Railway Data Report"
body = f"Hi good morning, Please find the attached Excel report of {yesterday_str} which has total result of {result_count}. Thank you."

attachment_path = full_path  # Full path to your Excel file

# Setup the yagmail client
yag = yagmail.SMTP(user=sender_email, password=app_password)

# Send the email
yag.send(
    to=receiver_emails,
    subject=subject,
    contents=body,
    attachments=attachment_path
)

print("Email sent successfully.")

Email sent successfully.


In [37]:
# ✅ Close browser
driver.quit()
print("tab is close now")

tab is close now
